# Análisis de datos

En este capítulo mostraremos varios ejemplos de manipulación de datos usando `numpy` y `pandas`. No es necesario aprender nada de memoria. Lo importante es entender el tipo de procesos que podemos hacer usando estas herramientas y tener a mano una colección de ejemplos típicos para adaptarlos a nuestras necesidades.

## Números pseudoaleatorios y estadística elemental

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.rc('figure',figsize=(5,3))

Simulamos el lanzamiendo de 1000 dados. Queremos comprobar que los 6 resultados son igualmente probables.

In [ ]:
dados = np.random.randint(1,6+1,1000)

In [ ]:
dados

El histograma de resultados debe mostrar una distribución aproximadamente uniforme.

In [ ]:
plt.hist(dados)

Las opciones por omisión no son adecuadas para este problema, quedan cajas vacías. Normalmente es conveniente especificar los extremos de las cajas ("bins"), para que el histograma quede mejor:

In [ ]:
plt.hist(dados,bins=np.arange(0.5,7),edgecolor='black');

Simulemos ahora 100 lanzamientos de 3 dados. Para ello generamos una matriz $100 \times 3$  de enteros pseudoaleatorios entre 1 y 6.

In [ ]:
dados = np.random.randint(1,6+1,(100,3))
dados[:10]

Para calcular la puntuación total en cada tirada sumamos los elementos de la matriz a lo largo de las filas:

In [ ]:
s = np.sum(dados,axis=1)
s

El histograma de resultados ya no es uniforme, empieza a parecerse a una normal.

In [ ]:
plt.hist(s,bins=np.arange(2.5,19), edgecolor='black');

La distribución acumulada empírica es otra forma de ver el tipo de aleatoriedad de unos datos. Se consigue simplemente ordenando la muestra.

In [ ]:
plt.plot(sorted(s),np.arange(len(s))/len(s));

Generemos ahora 100 números pseudo-aleatorios normalmente distribuidos con media 2 y desviación típica 3.

In [ ]:
r = 2+3*np.random.randn(100)

plt.hist(r,10);

Podemos compararar el histograma y distribución acumulada empírica de estos datos con la densidad y la distribución acumulada teóricas, que tomamos del módulo `scipy.stats`.

In [ ]:
from scipy.stats import norm

x = np.linspace(-5,10,100)

plt.figure(figsize=(10,3))
plt.subplot(1,2,1)
plt.hist(r,10,density=True,edgecolor='black');
plt.plot(x,norm.pdf(x,2,3));

plt.subplot(1,2,2)
plt.plot(np.sort(r),np.arange(1,101)/100);
plt.plot(x,norm.cdf(x,2,3));

`numpy` proporciona varias funciones estadísticas. Se pueden expresar como propiedades del array o como funciones normales. Elige la forma que más te guste:

In [ ]:
r.mean()

In [ ]:
np.mean(r)

In [ ]:
r.max()

In [ ]:
min(r)

In [ ]:
r.std()

In [ ]:
np.median(r)

Finalmente, supongamos que queremos saber cómo se transforma una distribución uniforme entre cero y uno con la función $f(x)=x^2$. La densidad de probabilidad resultante se puede obtener de forma analítica, pero también podemos visualizar el resultado fácilmente aplicando la función a una muestra aleatoria.

In [ ]:
x = np.random.rand(10000)
plt.hist(x,np.linspace(0,1,20));

In [ ]:
y = x**2
plt.hist(y, np.linspace(0,1,20));

## Lectura desde archivo

El módulo `pandas` proporciona el tipo "dataframe", muy utilizado en análisis de datos. Permite leer conjuntos de datos almacenados en archivos que pueden estar incluso en una máquina remota.

El siguiente archivo contiene datos astronómicos tomados de https://ned.ipac.caltech.edu/.

In [ ]:
import pandas as pd

In [ ]:
# Si estamos en jupyterlite lo leemos de la carpeta local datos, y si no de una máquina remota.

try:
    df = pd.read_table('https://robot.inf.um.es/material/data/ConstanteHubbleDatos-1.txt', sep='\s+', comment='#')
except:
    df = pd.read_table('datos/hubble.txt', sep='\s+', comment='#')    

df

Un experimento interesante es ajustar un modelo lineal a un conjunto de observaciones. Definimos una función auxiliar para dibujar cómo queda una recta cualquiera:

In [ ]:
# extraemos las columnas de las variables que nos interesan
x = df.Magnitud.values
y = df['V(km/s)'].values
n = len(x)

# dibuja una recta con coeficientes a,b entre x1 y x2
def abline(a,b,x1,x2):
    x = np.linspace(x1,x2,10)
    plt.plot(x,a*x+b)


def fun(a = 1200, b=0):
    plt.figure(figsize=(5,5))
    plt.plot(x,y,'.',markersize=10)
    plt.grid(); plt.xlabel('Magnitud'); plt.ylabel('velocity'); 
    plt.title('Redshift data');
    abline(a, b, 0,20)
    plt.axis([11,20,-10000,40000])

Por ejemplo:

In [ ]:
fun(1100,500)

Por supuesto, tiene más sentido encontrar automáticamente la "recta de regresión", que minimiza el error cuadrático. Lo vamos a hacer con el módulo `scikit-learn`.

In [ ]:
# !pip install scikit-learn

from sklearn import linear_model

In [ ]:
model = linear_model.LinearRegression()
model.fit(x.reshape(n,1),y.reshape(n,1))  # hace falta convertir los vectores en matrices

In [ ]:
model.coef_

In [ ]:
model.intercept_

In [ ]:
fun(model.coef_[0][0], model.intercept_[0])

Habría que evaluar la calidad del modelo obtenido sobre muestras independientes para ver si tiene sentido usarlo para predecir casos futuros.

## Ciudades

El módulo pandas puede leer hojas de cálculo de Excel. Como ejemplo vamos a hacer unos cuantos experimentos con las ciudades de España.

In [ ]:
# (obsoleto, 2010)
# https://www.businessintelligence.info/varios/longitud-latitud-pueblos-espana.html

try: 
    # !pip install xlrd
    df = pd.read_excel('https://robot.inf.um.es/material/data/listado-longitud-latitud-municipios-espana.xls',skiprows=2)
except:
    %pip install xlrd
    df = pd.read_excel('datos/municipios.xls',skiprows=2)

df.head()

Tenemos un "dataframe" (tabla de datos) con los más de 8000 municipios. Podemos seleccionar los casos (filas) que cumplan una condición, y los atributos deseados (columnas). Finalmente podemos ordenar por cualquier criterio.

In [ ]:
df[df.Habitantes > 200000][['Población','Habitantes']].sort_values(by='Habitantes', ascending=False)

Como primer ejemplo vamos a dibujar la posición de las ciudades más grandes.

In [ ]:
sel = df[(df.Habitantes > 250000) & (df.Comunidad != 'Canarias')]

x   = sel.Longitud.values
y   = sel.Latitud.values
pob = sel.Población.values

In [ ]:
plt.figure(figsize=(6,6))
plt.plot(x,y,'.',markersize=15)
for k in range(len(pob)):
    plt.text(x[k]+0.2,y[k],pob[k],fontsize=10)
plt.xlabel('longitud'); plt.ylabel('latitud');

Ten en cuenta que las coordenadas esféricas representadas en un plano producen una deformación.

## Ciudades más alejadas

¿Cuál es la pareja de ciudades españolas  más alejadas entre sí, dentro de la península? (Podemos considerar solo las más grandes, con más de 10K habitantes por ejemplo.)

Para resolver este problema necesitamos dos cosas:

- una función que convierta posiciones GPS en puntos sobre la superficie terrestre (suponemos una esfera y altitud común) para calcular la distancia correctamente.


- hacer un bucle doble para calcular las distancias entre todas las parejas de ciudades. 

En primer lugar vamos a crear una función auxiliar para extraer las coordenadas gps de una ciudad dada.

In [ ]:
df[df.Población=='Toledo']

In [ ]:
df[df.Población=='Toledo'][['Latitud','Longitud']].iloc[0]

Viendo cómo funciona la selección de filas y columnas y tras unas cuantas pruebas, elegimos esa definición:

In [ ]:
def gps(ciudad):
    return np.array(df[df.Población==ciudad][['Latitud','Longitud']].iloc[0])

gps('Toledo')

Para calcular la distancia entre dos puntos gps convertimos las coordenadas esféricas a vectores 3D cartesianos y luego vemos el ángulo que forman.

In [ ]:
def tovec(p):
    la,lo = np.radians(p)
    z = np.sin(la)
    x = np.cos(la) * np.cos(lo)
    y = np.cos(la) * np.sin(lo)
    return np.array([x,y,z])

RT = 6371

def geodist(p,q):
    v1 = tovec(p)
    v2 = tovec(q)
    x = v1 @ v2
    if abs(x) > 1:
        return 0
    a = np.arccos(x)
    return RT*a


geodist(gps('Murcia'),gps('Cartagena'))

Ya estamos en condiciones de encontrar al máxima distancia. Primero "filtramos" las ciudades, quitando las que están fuera de la península o son pequeñas.

In [ ]:
# OJO, hay errores en la hoja excel en las coordenadas gps de algún pueblo pequeño.
sel = df[  (df.Comunidad !='Canarias') 
         & (df.Comunidad != 'Islas Baleares') 
         & (df.Comunidad != 'Ceuta y Melilla') 
         & (df.Habitantes>10000)]

n = len(sel)
print(n)
pob = sel.Población.values
x   = sel.Latitud.values
y   = sel.Longitud.values
r = np.array([x,y]).T
r[:3]

Hemos juntado x e y en una matriz r, de modo que r[k] nos da las coordenadas gps de la ciudad k-esima.

Creamos una lista de tuplas con todos los pares de ciudades y sus distancias.

In [ ]:
dists = [ (geodist(r[k], r[j]), pob[k], pob[j])
            for k in range(n)
            for j in range(n)
            if k>j ]

In [ ]:
dists[:5]

Y finalmente ordenamos la lista de tuplas. La ordenación de objetos que no son simples números se hace atendiendo a su primer elemento.

In [ ]:
sorted(dists,reverse=True)[:10]

Podemos resolver el mismo problema con otra selección de ciudades:

In [ ]:
sel = df[(df.Comunidad =='Murcia')]

n = len(sel)
print(n)
pob = sel.Población.values
x   = sel.Latitud.values
y   = sel.Longitud.values
r = np.array([x,y]).T

dists = [ (geodist(r[k], r[j]), pob[k], pob[j])
            for k in range(n)
            for j in range(n)
            if k>j ]

In [ ]:
sorted(dists,reverse=True)[:10]

Nota: es posible calcular una matriz de distancias entre parejas de puntos usando `numpy`, lo que es mucho más eficiente que el bucle doble explícito de Python. Pero no merece la pena complicar el ejercicio.

## Zipf's law

Aprovechando que tenemos el número de habitantes de las ciudades vamos a mostrar su histograma:

In [ ]:
plt.figure(figsize=(10,3))
plt.hist(df[(df.Habitantes>100)].Habitantes,log=True,bins=30);

Tiene una forma muy característica, completamente distinta de las distribuciones normales (con forma de campana) que se observan en otro tipo de procesos aleatorios. El histograma en escala logarítmica parece que se comporta mejor:

In [ ]:
plt.figure(figsize=(10,3))
plt.hist(df[(df.Habitantes>100)].Habitantes,log=True,bins=np.logspace(1, 7, 50));plt.xscale('log');

Pero lo que realmente es indicativo es la distribución acumulada empírica:

In [ ]:
plt.figure(figsize=(10,3))
x = sorted(np.array(df[df.Habitantes>100].Habitantes))
k = np.arange(len(x))/len(x)
plt.plot(x,k); plt.xlim(-1000,10000);

Tiene toda la pinta de ser una distribución "scale free" o  "[power law](https://en.wikipedia.org/wiki/Power_law)". Si mostramos en escalas logarítmicas el tamaño de cada ciudad frente a su número de orden obtenemos una dependencia esencialmente lineal, cuya pendiente es el parámetro característico de la distribución.

In [ ]:
dfs = df[df.Habitantes>1000].sort_values('Habitantes',ascending=False)

names = list(dfs.Población)
x = list(dfs.Habitantes)
plt.figure(figsize=(12,6))
plt.plot(1+np.arange(len(x)), x ,'.-',lw=1,markersize=5);
for k in range(20):
    plt.text(1+k,x[k],names[k],rotation=45,horizontalalignment='left',verticalalignment='bottom',fontsize=8)
plt.xscale('log'); plt.yscale('log');

Lo mismo ocurre en un subconjunto (al menos en un cierto rango de tamaños).

In [ ]:
dfs = df[df.Comunidad=='Murcia'].sort_values('Habitantes',ascending=False)

names = list(dfs.Población)
x = list(dfs.Habitantes)
plt.figure(figsize=(12,6))
plt.plot(1+np.arange(len(x)), x ,'.-',lw=1,markersize=5);
for k in range(len(x)):
    plt.text(1+k,x[k],names[k],rotation=45,horizontalalignment='left',verticalalignment='bottom',fontsize=8)
plt.xscale('log'); plt.yscale('log');

Este tipo de distribución se observa en muchos otros fenómenos. Por ejemplo, en la frecuencia de las palabras de un idioma:

In [ ]:
#https://en.wiktionary.org/wiki/User:Matthias_Buchmeier/Spanish_frequency_list-1-5000
data = open('datos/palabras.txt').read().split(' ')
freqs = np.array(data[0::2]).astype(float)
words = data[1::2]

ks = np.arange(1,1+len(freqs))

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(ks,freqs,'.-');
for k in range(25):
    plt.text(1+k,freqs[k],words[k],rotation=45,horizontalalignment='left',verticalalignment='bottom',fontsize=8)
plt.xscale('log'); plt.yscale('log');